### Pydantic
pydantic models provide the richest feature set with field validation, descriptions and nested structures

In [2]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model =init_chat_model("groq:qwen/qwen3-32b")

In [3]:
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000017B714C94D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017B7162D5D0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")



In [5]:
model_with_structured_output = model.with_structured_output(Movie)
model_with_structured_output

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000017B714C94D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017B7162D5D0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', 'dire

In [6]:
model_with_structured_output.invoke("Provide details of Movie Bahubali2")

Movie(title='Baahubali 2: The Conclusion', year=2017, director='S.S. Rajamouli', rating=8.0)

In [7]:
model_with_structured_output.invoke("Provide details of Movie RRR")

Movie(title='RRR', year=2022, director='S.S. Rajamouli', rating=9.5)

In [8]:
class Movie(BaseModel):
    """A Movie with details"""
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")

model_with_structure = model.with_structured_output(Movie,include_raw=True)

response = model_with_structure.invoke("Provide Details about Movie Bahubali")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking for details about the movie Bahubali. Let me check the tools provided. There's a Movie function that requires title, year, director, and rating. I need to fill these in for Bahubali. I remember that Bahubali was directed by S.S. Rajamouli. The original release year was 2015. The rating is a bit tricky, but I think it has a high rating, maybe around 8.5 or 9. Let me confirm. Yes, on IMDb it's around 8.3. So I'll use that. Let me structure the JSON with the required parameters.\n", 'tool_calls': [{'id': 'a272atn28', 'function': {'arguments': '{"director":"S.S. Rajamouli","rating":8.3,"title":"Bahubali","year":2015}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 189, 'prompt_tokens': 229, 'total_tokens': 418, 'completion_time': 0.335441074, 'completion_tokens_details': {'reasoning_tokens': 135}, 'prompt_time': 0.009200851, 'prompt_tokens_detai

### Nested Structure

In [9]:
class Actor(BaseModel):
    name:str
    role:str


class MovieDetails(BaseModel):
    title:str
    year:str
    cast:list[Actor]
    genres:list[str]
    budget:float|None = Field(None,description="Budget in millions USD")


model_with_structure  = model.with_structured_output(MovieDetails)
response = model_with_structure.invoke("Provide Details of Movie Bahubali2")
response

MovieDetails(title='Bahubali 2', year='2017', cast=[Actor(name='Prabhas', role='Bahubali'), Actor(name='Rana Daggubati', role='Bhallala Deva'), Actor(name='Anushka Shetty', role='Avanthika'), Actor(name='Tamannaah Bhatia', role='Devasena'), Actor(name='Sathyaraj', role='Somdev')], genres=['Action', 'Adventure', 'Fantasy'], budget=100.0)

### Typed Dict

This provides a simpler alternative using python's built-in typing, ideal when you don't need run time validation

In [10]:
from typing_extensions import TypedDict,Annotated

class MovieDict(TypedDict):
    """A Movie with details)"""
    title:Annotated[str,...,"The title of the movie"]
    year:Annotated[int,...,"This year movie was released"]
    director:Annotated[str,...,"The director of the movie"]
    rating:Annotated[float,...,"The movies rating out of 10"]

model_with_typed_dict = model.with_structured_output(MovieDict)
response = model_with_typed_dict.invoke("Please provide the details of movie Bahubali")

response


{'director': 'S. S. Rajamouli',
 'rating': 8,
 'title': 'Bahubali: The Beginning',
 'year': 2015}

In [11]:
class Actor(TypedDict):
    name:str
    role:str


class MovieDetails(TypedDict):
    title:str
    year:str
    cast:list[Actor]
    genres:list[str]
    budget:float|None = Field(None,description="Budget in millions USD")


model_with_structure  = model.with_structured_output(MovieDetails)
response = model_with_structure.invoke("Provide Details of Movie Bahubali2")
response

{'budget': 100000000,
 'cast': [{'name': 'Prabhas', 'role': 'Bahubali'},
  {'name': 'Rana Daggubati', 'role': 'Bhallala Deva'},
  {'name': 'Tamannaah', 'role': 'Devasena'},
  {'name': 'Sudeep', 'role': 'Bhairava'}],
 'genres': ['Action', 'Adventure', 'Drama'],
 'title': 'Bahubali2',
 'year': '2017'}